### Spotify EDA
#### Clean Up DB

Database Cleanup Actions:
- Make new playlist table to store deleted entries

- iterate over all playlist items in batches of 50
- Check that artist link is valid
- If it returns something else - overwrite initial uri with the returned uri
- Check that artist is in artist table - if not load new artist - update popularity
- Check that track link is valid - if not valid, move to deleted entries table
- If valid, check that uri is in the track table - update popularity of track and artist


In [29]:
import warnings
warnings.filterwarnings('ignore')
%config InlineBackend.figure_format ='retina'

In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import os
import sys
import json
import seaborn as sns
import time
from time import sleep

from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Table, Column, Integer, String, Float, MetaData, and_, or_
from sqlalchemy import create_engine
import sqlite3
from sqlalchemy.orm import sessionmaker
from sqlalchemy import exc

sys.path.append('../')
from spotify_api import get_spotify_data, get_tracks, get_artists, get_audiofeatures
from spotify_database import get_session, display_time
from spotify_utils import Table_Generator, List_Generator, pickle_load, pickle_save

%matplotlib inline

In [2]:
# !pip install ipywidgets 
# !jupyter nbextension enable --py widgetsnbextension
# !jupyter labextension install @jupyter-widgets/jupyterlab-manager

# %%capture
from tqdm import tqdm_notebook as tqdm


### Establish DataBase Connection

In [3]:
db_path = '../data/SpotifyDataSet/spotify_songs.db'

# Get sesion
session = get_session(db_path)
engine = create_engine('sqlite:///' + db_path)

# Get Songs class
Playlists = getattr(get_session, "Playlists")
Artists = getattr(get_session, "Artists")
Tracks = getattr(get_session, "Tracks")


In [4]:
# Create DB and Table if they don't exist


# connection = engine.connect() #creates db if it doesn't exist
# meta = MetaData()

# songs = Table(
#                 'songs', meta,
#                 Column('id', Integer, primary_key = True),
#                 Column('playlist_id', Integer),
#                 Column('pos', Integer), 
#                 Column('artist_name', String), 
#                 Column('track_uri', String),
#                 Column('artist_uri', String), 
#                 Column('track_name', String),
#                 Column('album_uri', String), 
#                 Column('duration_ms', String), 
#                 Column('album_name', String)
# )

# tracks = Table(
#                 'tracks', meta,
#                 Column('track_uri', String, primary_key = True),
#                 Column('artist_uri', String),
#                 Column('danceability', Float),
#                 Column('energy', Float),
#                 Column('key', Integer),
#                 Column('loudness', Float),
#                 Column('mode', Integer),
#                 Column('speechiness', Float),
#                 Column('acousticness', Float),
#                 Column('instrumentalness', Float),
#                 Column('liveness', Float),
#                 Column('valence', Float),
#                 Column('tempo', Float),
#                 Column('duration_ms', Integer),
#                 Column('time_signature', Integer)
# )

# artists = Table(
#                 'artists', meta,
#                 Column('artist_uri', String, primary_key = True),
#                 Column('followers', Integer),
#                 Column('genres', String),
#                 Column('artist_name', String),
#                 Column('artist_popularity', Integer)
# )

# meta.create_all(engine)

In [5]:
# session.rollback()
# session.commit()

### Establish Spotify API Connection

In [6]:
# Get URI for Artist, Track and Album for a song
rv = display_time(session.query(Playlists.track_name, 
                                Playlists.track_uri,
                                Playlists.artist_uri,
                                Playlists.album_uri).filter(Playlists.track_name=="Who Says").distinct().first)


Time to Execute: 0.0 seconds


In [7]:
rv.artist_uri

'spotify:artist:0hEurMDQu99nJRq8pTxO14'

In [8]:
get_spotify_data(db_uri=rv.artist_uri, key='genres')

Setting credentials
token():INFO:   Getting initial token
token():INFO:   Token refreshed


['neo mellow', 'pop', 'pop rock', 'singer-songwriter']

In [9]:
# def build_track_string(uri_list:list) -> str:
#     rv_str = ''
    
#     for u in uri_list:
#         t_type, t_id = parse_spotify_url(u)
#         if t_type != "track":
#             print("ERR: Only 'track'-type uri's have features: {}".format(u))
#             return False
        
#         if rv_str == '':
#             rv_str += t_id
#         else:
#             rv_str += ',' + t_id
            
#     return rv_str

In [10]:
def load_tracks(uri_list, db_path, batch_size:int=100):
    """
    Loads a list of uris into the database.
    Returns a list of uris that were loaded
    If any item fails, no items from list are loaded.
    """
    
    engine = create_engine('sqlite:///' + db_path)
    Session = sessionmaker(bind = engine)
    session = Session()
    
    batch_gen = List_Generator.batch_generator(uri_list, batch_size=batch_size)

    loaded_uris = []
    for uri_batch in batch_gen:

        sp_tracks = get_audiofeatures(uri_batch)

        for t in sp_tracks:
            try:
                if t == None:
                    loaded_uris.append(None)

                else:
                    session.add(Tracks( track_uri           = t.get('uri'),
                                        artist_uri          = t.get('artists')[0].get('uri'),
                                        danceability        = t.get('danceability'),
                                        energy              = t.get('energy'),
                                        key                 = t.get('key'),
                                        loudness            = t.get('loudness'),
                                        mode                = t.get('mode'),
                                        speechiness         = t.get('speechiness'),
                                        acousticness        = t.get('acousticness'),
                                        instrumentalness    = t.get('instrumentalness'),
                                        liveness            = t.get('liveness'),
                                        valence             = t.get('valence'),
                                        tempo               = t.get('tempo'),
                                        duration_ms         = t.get('duration_ms'),
                                        time_signature      = t.get('time_signature')
                                      ) 
                               )
                    loaded_uris.append(t.get('uri'))

            except Exception as e:
                print(e)
                print("Problem happened at uri: {}".format(t))
                print("Graceful shutdown of session...")
                session.rollback()
                print("Closing session...")
                session.close()
                return [None]

    session.commit()
    session.close()
    return loaded_uris
    

In [11]:
def load_artists(artists_uri_list, db_path, batch_size:int=50):
    """
    Loads new artists into the Artits table.
    Can send a long list.  It will batch into batches of batch_size.
    Loads as many as it can before failing.
    """
    
    if type(artists_uri_list) != list:
        print("load_artsist() needs a list. '{}' type provided.".format(type(artists_uri_list)))
        return 0
    
    engine = create_engine('sqlite:///' + db_path)
    Session = sessionmaker(bind = engine)
    session = Session()
 
    loaded_uris = []
    
    batch_gen = List_Generator(artists_uri_list, batch_size=batch_size).batch_generator()
    
    for uri_batch in batch_gen:
    
        artists = get_artists(uri_batch)

        for a in artists:

            if a == None:
                loaded_uris.append(None)
                continue

            session.add(Artists( artist_uri          = a.get('uri'),
                                 followers           = a.get('followers').get('total'),
                                 genres              = ",".join(a.get('genres')),
                                 artist_name         = a.get('name'),
                                 artist_popularity   = a.get('popularity')
                               )
                       )

        try:
            session.commit()
            loaded_uris.append(a.get('uri'))
            
        except Exception as e:
            print(e)
            session.rollback()

            # commit individually
            for i, a in enumerate(artists):
                session.add(Artists( artist_uri          = a.get('uri'),
                                     followers           = a.get('followers').get('total'),
                                     genres              = ",".join(a.get('genres')),
                                     artist_name         = a.get('name'),
                                     artist_popularity   = a.get('popularity')
                                   )
                           )
                try:
                    session.commit()
                    loaded_uris.append(a.get('uri'))
                    
                except Exception as e:
                    print("Artist Load ERR:{}   {} - {}".format(i, a.get('uri'), a.get('name') ))
                    session.rollback()

    session.close()
    return loaded_uris

### DB Cleanup

Database Cleanup Actions:
- Make new playlist table to store deleted entries

- iterate over all playlist items in batches of 50
- Check that artist link is valid
- If it returns something else - overwrite initial uri with the returned uri
- Check that artist is in artist table - if not load new artist - update popularity
- Check that track link is valid - if not valid, move to deleted entries table
- If valid, check that uri is in the track table - update popularity of track and artist

In [12]:
# Check that artists exist in Artist table - load, if they don't
# returns the artist db entries
def get_db_artists(artist_uri_list, load_missing:bool=False, debugging:bool=False) -> list:
    """
    Checks that a list of artists exists in the Artists
    table and will load any that don't exist if load_missing=True.
    Returns list of db records for artists.  
    If not found, None entry is included in list.
    """
    session = get_session(db_path)
    
    db_artist_list = []
    try:
        for a in artist_uri_list:
            db_artist = session.query(Artists).filter(Artists.artist_uri==a).first()
            if db_artist == None: # if not in table
                if load_missing:
                    if debugging:
                        print("Loading missing artist: {}".format(a))
                    loaded_uris = load_artists([a], db_path)
                    if loaded_uris[0] == None:
                        db_artist_list.append(None)
                    else:
                        # now get the db isting of the artist
                        db_artist = session.query(Artists).filter(Artists.artist_uri==a).first()
                        db_artist_list.append(db_artist)
                else:
                    db_artist_list.append(None)
    except Exception as e:
        print("get_db_artists(): ERR at laoding uri: {}".format(a))
        print(e)
        
    
    session.close()
    
    if debugging:
        none_idxs = np.where(np.array(db_artist_list))
        if len(none_idxs)>0:
            print("Note uris that are not in DB. Returned as 'None' is list:")
            for i in none_idxs:
                print("\t{}".format(artist_uri_list[i]))
    
    return db_artist_list

In [13]:
# Check that artists exist in Artist table - load, if they don't
# returns the artist db entries
def get_db_tracks(track_uri_list, load_missing:bool=False, debugging:bool=False) -> list:
    """
    Checks that a list of tracks exists in the Tracks
    table and will load any that don't exist if load_missing=True.
    Returns list of db records for tracks.  
    If not found, None entry is included in list.
    """
    session = get_session(db_path)
    
    db_track_list = []
    try:
        for t in track_uri_list:
            db_track = session.query(Tracks).filter(Tracks.track_uri==t).first()
            if db_track == None:
                if load_missing:
                    if debugging:
                        print("Loading missing track: {}".format(t))
                    loaded_uris = load_tracks([t], db_path)
                    if loaded_uris[0] == None:
                        db_track_list.append(None)
                    else:
                        # now get the db listing of the track
                        db_track = session.query(Tracks).filter(Tracks.track_uri==t).first()
                        db_track_list.append(db_track)
                else:
                    db_track_list.append(None)
    except Exception as e:
        print("get_db_tracks() ERR at loading uri: {}".format(t))
        print(e)
    
    session.close()
    
    if debugging:
        none_idxs = np.where(np.array(db_track_list))
        if len(none_idxs)>0:
            print("Note uris that are not in DB. Returned as 'None' is list:")
            for i in none_idxs:
                print("\t{}".format(track_uri_list[i]))
        
    return db_track_list

In [14]:
def update_artist_table(artist_uris:list=None, batch_size:int=50):
    """
    Updates the artist followers, popularity and genres in the Artist table.
    If 'artist_uris' argument is blank, updates all.
    If list is provided, updates only the items in the list and adds them if missing.
    """
    main_session = get_session(db_path)
    batch_gen = ''
    if artist_uris != None:
        db_artists = get_db_artists(artist_uris, load_missing=True) # creates entries if missing
        if len(db_artists) == 0:
            print("No matching URIs in DB.  No artists updated.")
            return
        batch_gen = List_Generator(db_artists, batch_size=batch_size).batch_generator()
    else:
        query = main_session.query(Artists)
        batch_gen = Table_Generator(query=query, batch_size=batch_size).batch_generator()
    
    try:
        for db_artists in batch_gen:

            # get spotify artists
            db_artist_uris = [a.artist_uri for a in db_artists]
            sp_artists = get_artists(db_artist_uris)
            if sp_artists == False:
                raise Exception("Unable to get data from Spotify.  Service may be down.")

            # update each artist with spotify data
            for db_artist, sp_artist in zip(db_artists, sp_artists):
                db_artist.followers = sp_artist.get("followers").get("total")
                db_artist.artist_popularity = sp_artist.get("popularity")
                db_artist.genres = ",".join(sp_artist.get('genres'))

    #             print(sp_artist.get("followers").get("total"), sp_artist.get("popularity"), ",".join(sp_artist.get('genres')))
    except Exception as e:
        print(e)
        print("Graceful session termination...")
        main_session.rollback()
        main_session.close()
        return None
    
    print("Committing Session ...")
    main_session.commit()
    print("Closing Session ...")
    main_session.close()
    

In [17]:
# # Takes about 15 minutes
update_artist_table()

Creating Table Generator:
	batch size :  50
	length     :  296014
	num batches:  5921



Committing Session ...
Closing Session ...


In [15]:
def update_tracks_table(track_uris:list=None, batch_size:int=50):
    """
    Updates the tracks table popularity.
    If argument is blank, updates all table records.
    If argument is provided, updates only the items in the list and adds them if missing.
    """
    main_session = get_session(db_path)
    batch_gen = ''
    artists_not_in_spotify = []
    if track_uris != None:
        #make generator from a list
        db_tracks = get_db_tracks(track_uris) # creates entries if missing
        if len(db_tracks) == 0:
            print("No matching URIs in DB.  No tracks updated.")
            return
        
        batch_gen = List_Generator(db_tracks, batch_size=batch_size).batch_generator()
    else:
        # make generator from database
#         query = main_session.query(Tracks)
        query = main_session.query(Tracks).filter(Tracks.track_popularity==None)
        batch_gen = Table_Generator(query=query, batch_size=batch_size).batch_generator()
    
    for db_tracks in batch_gen:

        # get spotify tracks
        db_track_uris = [t.track_uri for t in db_tracks]
        sp_tracks = get_tracks(db_track_uris)
        if sp_tracks == False:
            continue
        
        # see if spotify returns some as None - remove them from the update lists
        none_idxs = np.where(np.array(sp_tracks)==None)[0]
        
        try:
            if len(none_idxs) > 0:
                none_idxs[::-1].sort() # sort them descending to pop off the end first
                for i in none_idxs:
#                     print("index: {}  db_tracks: {}   sp_tracks: {}".format(i,len(db_tracks),len(sp_tracks)) )
                    db_tracks.pop(i)
#                     db_track_uris.pop(i)
                    sp_tracks.pop(i)
                    artists_not_in_spotify.append(db_track_uris.pop(i))

            if len(db_tracks)!= len(sp_tracks):
                print("List length error.  db_tracks:{} sp_tracks:{}".format(db_tracks, sp_tracks))
                main_session.rollback()
                main_session.close()
                return
        except Exception as e:
            print(e)
            print("Closing.  Check rv for values on error.")
            main_session.close()
            return [db_tracks, sp_tracks, db_track_uris, artists_not_in_spotify]
            
            
        
        # update each track with spotify data
        for db_track, sp_track in zip(db_tracks, sp_tracks):
            if db_track==None or sp_track==None:
                continue
            db_track.track_popularity = sp_track.get("popularity")
            db_track.artist_uri = sp_track.get("artists")[0].get("uri")
    
            main_session.commit()
        
    main_session.close()
    
    return artists_not_in_spotify

In [37]:
# # takes about 2hrs
missing_tracks_in_spotify = update_tracks_table()

Creating Table Generator:
	batch size :  50
	length     :  584
	num batches:  12


In [16]:
# save pickle file of missing track uris
# pickle_save(missing_tracks_in_spotify, 'missing_tracks_in_spotify.pkl')

In [17]:
def delete_tracks(track_uris:list, table_name, batch_size:int=200) -> list:
    """
    Deletes tracks from the table_name provided
    """
    # if num tracks is < batch_size, make batch_size same as num track_uris
    if len(track_uris) < batch_size:
        batch_size = len(track_uris)
    
    # Setup generators
    session = get_session(db_path)
    batch_list_gen = List_Generator(track_uris, batch_size=batch_size).batch_generator()
    
    for batch_track_uris in batch_list_gen:
        db_records = session.query(table_name).filter(table_name.track_uri.in_(batch_track_uris)).delete()
        session.commit()
    
    session.close()
    
    

In [18]:
def clean_up_db(delete:bool=False, batch_size:int=50):
    """
    Check to make sure that the artist uri is current.
    Make sure entries have valid track and artist uris.
    Removes items where uri is not valid.
    """
    main_session = get_session(db_path)
    query = main_session.query(Playlists)
    batch_gen = Table_Generator(query=query, batch_size=batch_size).batch_generator()

    playlist_ids_to_delete = []
    artists_to_update = []
    
    try:
        for batch in batch_gen:

            # build a list of track uris, artist uris
            uris = [(r.track_uri, r.artist_uri) for r in batch]
            uris = np.array(uris)
            db_track_uris = uris[:,0]
            db_artist_uris = uris[:,1]


            # TRACKS - adds if they dont exists - includes None's where invalid
            db_tracks = np.array(get_db_tracks(db_track_uris.tolist()))

            # ARTISTS - adds if they dont exists - includes None's where invalid
            db_artists = np.array(get_db_artists(db_artist_uris.tolist(), load_missing=True))

            # Check if any tracks/artists are None - remove them from db and lists we made
            for db_cat in (db_tracks, db_artists):
                none_idxs = np.where(db_cat==None)[0]
                if len(none_idxs)>0:
                    # remove from db, batch, sp_tracks and sp_artists
                    for idx in none_idxs:
                        if delete:
                            main_session.delete(batch[idx])
                        playlist_ids_to_delete.append(batch[idx].get('id'))
                        batch.pop(idx)
                        db_tracks.pop(idx)
                        db_artists.pop(idx)

                    # remove from db_track_uris, db_artist_uris, 
                    db_track_uris  = np.delete(db_track_uris, none_idxs)
                    db_artist_uris = np.delete(db_artist_uris, none_idxs)


            # make sure we don't have any list mis-matches
            if len(db_artist_uris) != len(batch):
                raise Exception("clean_up_db() ERR: list length error: db_artists={}  batch={}".format(len(db_artist_uris), len(batch))) 

            # iterate over remaining records in batch
            # update artist if changed - spotify changes artist uris sometimes
            for r, record in enumerate(batch):
                # check if artist uri has been updated - if so update in playlists table
                db_playlist_artist_uri = record.artist_uri
                db_artist_uri = db_artist_uris[r]

                if  db_playlist_artist_uri != db_artist_uri:
                    print("Need to update artist from: {}  to: {}".format(db_playlist_artist_uri, db_artist_uri))
                    artists_to_update.append((db_playlist_artist_uri, db_artist_uri))
                    if delete:
                        record.artist_uri = db_artist_uri
    except Exception as e:
        print(e)
        print("Graceful rollback ...")
        main_session.rollback()
        print("Session close ...")
        main_session.close()
        return artists_to_update, playlist_ids_to_delete

    print("Session commit ...")
    main_session.commit()
    print("Session close ...")
    main_session.close()
    
    return artists_to_update, playlist_ids_to_delete
    

In [19]:
# takes days to execute
# artists_to_update, playlist_ids_to_delete = clean_up_db()

### Missing Tracks Review and Updates

In [16]:
eric_missing_tracks = pickle_load('eric_bad_track_uris.pkl')

In [17]:
mark_missing_tracks = pickle_load('mark_missing_tracks_in_spotify.pkl')

In [18]:
len(eric_missing_tracks), len(mark_missing_tracks)

(584, 556)

In [19]:
in_both_lists = list(set(eric_missing_tracks) & set(mark_missing_tracks))

In [20]:
len(in_both_lists)

556

In [21]:
in_eric_not_in_mark = set(eric_missing_tracks) - set(mark_missing_tracks)
in_mark_not_in_eric = set(mark_missing_tracks) - set(eric_missing_tracks)


In [22]:
len(in_eric_not_in_mark), len(in_mark_not_in_eric)

(28, 0)

### Check a Missing Tracks List
Make sure that all of the missing tracks return None from Spotify

In [20]:
def check_tracks_for_none(track_uris:list) -> np.array:
    """
    Checks that a list of artists returns None.
    Returns list of valid artist urls that were in the list.
    """
    batch_gen = List_Generator(track_uris, batch_size=50).batch_generator()

    orig_tracks = []
    sp_tracks = []
    for batch in batch_gen:
        batch_sp_tracks = get_tracks(batch)
        for t in batch_sp_tracks:
            if t==None:
                sp_tracks.extend(["None"])
            else:
                sp_tracks.extend([t.get('uri')])

        orig_tracks.extend(batch)

    idxs = np.where(np.array(sp_tracks)!="None")

    if len(idxs[0])>0:
        print("The following tracks were in list but returned sportify values:")
        for i in idxs[0]:
            print("list: {}  spotify: {}".format(orig_tracks[i], sp_tracks[i]))
        return np.array(sp_tracks)[idxs]
    else:
        print("All provided tracks returned 'None' from Spotify!")
    
    return None

In [47]:
sp_tracks = check_tracks_for_none(eric_missing_tracks)

Creating List Generator:
	batch size :  50
	length     :  584
	num batches:  12


token():INFO:   Token refreshed

All provided tracks returned 'None' from Spotify!


### Remove the Missing Tracks from Table
specifiy Tracks or Playlists in the TABLE argument

In [27]:
def remove_none_tracks(track_uris:np.array, TABLE, batch_size:int=50) -> list:
    """
    Removes a list of uris from the Tracks and Playlists tables.
    First checks that items truly return None from Spotify before deleting.
    """
    
    print("Preparing to delete records from table: \n{}".format(TABLE))
    
    # First check that they are all "None" in Spotify
    sp_results = check_tracks_for_none(track_uris)
    if sp_results != None:
        print("Some URIS were valid in Spotify.  No deletions made.")
        return sp_results
    
    removed_uris = []
    
    # get Table entries
    print("Getting records to delete from database...")
    main_session = get_session(db_path)
    query = main_session.query(TABLE).filter(TABLE.track_uri.in_(track_uris))
    batch_gen = Table_Generator(query=query, batch_size=batch_size, name="deleting tracks progress").batch_generator()
    
    # iterate batches and delete entries
    del_count = 0
    try:
        print("Deleting records ...")
        for batch in batch_gen:
            for item in batch:
                main_session.delete(item)
                main_session.commit()
                del_count += 1
            main_session.commit()
        
    except Exception as e:
        print (e)
        print ("ERR: Exception ocurred.  Rolling back.  No deletes made.")
        print ("ERR: Error ocurred when handling: {}",item.track_uri)
        main_session.rollback()
        main_session.close()
        return
    
#     print("Committing deletions ...")
    main_session.commit()
    print("Done.  Deleted {} records.".format(del_count))
    main_session.close()
    return 

In [25]:
missing_tracks = pickle_load('eric_bad_track_uris.pkl')

In [28]:
# Remove tracks from Playlists table
remove_none_tracks(missing_tracks, Playlists) # completed on 11/7/2019 - takes 3hrs

Preparing to delete records from table: 
<class 'spotify_database.get_session.<locals>.Playlists'>
Creating List Generator:
	batch size :  50
	length     :  584
	num batches:  12



All provided tracks returned 'None' from Spotify!
Getting records to delete from database...
Creating Table Generator:
	batch size :  50
	length     :  50
	num batches:  2
Deleting records ...



Done.  Deleted 50 records.


In [29]:
# Remove tracks from Tracks table
remove_none_tracks(missing_tracks, Tracks) # completed on 11/7/2019

Preparing to delete records from table: 
<class 'spotify_database.get_session.<locals>.Tracks'>
Creating List Generator:
	batch size :  50
	length     :  584
	num batches:  12


token():INFO:   Token refreshed

All provided tracks returned 'None' from Spotify!
Getting records to delete from database...
Creating Table Generator:
	batch size :  10
	length     :  0
	num batches:  1
Deleting records ...



Done.  Deleted 0 records.


### Update mis-matched Artist Records
Some records in the db return an updated artist uri from Spotify.
These are updated in the Playlists and Artists Table

In [137]:
def check_mismatched_artists(artist_uris:dict):
    """
    Checks that dict of artist returns a new value from Spotify.
    Argument is in form {db_uri:spotify_uri}
    Returns a list of the incorrect entries.  If list has zero length, all are correct.
    """
    print("Validating mis-mathced entries...")
    # convert to numpy array [:,0] is db uris and [:,1] is new spotify uris
    np_artists = np.array([(k,v) for (k,v) in mismatched_artists.items()])
    db_artists, sp_artists = np_artists[:,0], np_artists[:,1]
    
    db_batch_gen = List_Generator(list(db_artists), batch_size=50).batch_generator()
    sp_batch_gen = List_Generator(list(sp_artists), batch_size=50).batch_generator()
    
    sp_uris = []
    for db_artist_batch, sp_artist_batch in zip(db_batch_gen, sp_batch_gen):
        sp_artists = get_artists(db_artist_batch)
        sp_artists_uris = [a.get("uri") for a in sp_artists]
        sp_uris.extend(sp_artists_uris)
        
        # compare sp_artist_batch and sp_artists -- they should be the same
        mismatch_idxs = np.where(sp_artist_batch!=sp_artists_uris)
        
        if len(mismatch_idxs[0])>0:
            for idx in mismatch_idxs[0]:
                print("ERR: for db track: {}".format(db_artist_batch[idx]))
                print("\t List said: {}  spotify says: {}",format(sp_artist_batch[idx], sp_artists_uris[idx]))
    
    incorrect_entries = np_artists[np.where(np.array(sp_uris)!=np_artists[:,1])]
    if len(incorrect_entries)==0:
        print("All mis-matched entries have been validated!")

    return incorrect_entries
    
    

In [156]:
def update_mismatched_artists(mismatched_artists:dict, TABLE):
    """
    Updates artist uri records in a table.
    First checks that supplied table is correct.
    Nothing returned.  Confirmation message sent to output.
    """
    
    incorrect_entries = check_mismatched_artists(mismatched_artists)
    if len(incorrect_entries)>0:
        print("There are incorrectly identified mismatches in the list.")
        print("No updates were made.  Correct list and retry.")
        return incorrect_entries
    
    # convert to numpy array [:,0] is db uris and [:,1] is new spotify uris
    np_artists = np.array([(k,v) for (k,v) in mismatched_artists.items()])
    db_artists, sp_artists = np_artists[:,0], np_artists[:,1]
    
    # get Table entries
    print("Getting records to update from database...")
    main_session = get_session(db_path)
    query = main_session.query(TABLE).filter(TABLE.artist_uri.in_(list(db_artists)))
    batch_gen = Table_Generator(query=query, batch_size=50).batch_generator()
    
    update_count = 0
    old_uri = ''
    try:
        print("Updating records ...")
        for batch in batch_gen:
            for item in batch:
                new_uri = mismatched_artists.get(item.artist_uri)
                if new_uri != None:
                    old_uri = item.artist_uri
                    item.artist_uri = new_uri
                    try:
                        main_session.commit()
                        update_count += 1
                    except exc.IntegrityError as e:
                        print("Entry already exists: {}  Deleting old record: {}".format(new_uri, old_uri))
                        main_session.rollback()
                        main_session.delete(item)
                        main_session.commit()
    
    
    except Exception as e:
        print (e)
        print ("ERR: Exception ocurred.  Rolling back.  No deletes made.")
        print ("ERR: Error ocurred when handling: {}".format(old_uri))
        main_session.rollback()
        main_session.close()
        return
    
    print("Updates complete!  {} records updated.".format(update_count))
    return
    
    

In [146]:
mismatched_artists = pickle_load('eric_mismatched_artist_uris.pkl')

In [160]:
# Update mismatched artists in the Playlists table
# update_mismatched_artists(mismatched_artists, TABLE=Playlists) # updated 11/7

Validating mis-mathced entries...
Creating List Generator:
	batch size :  50
	length     :  17
	num batches:  1
Creating List Generator:
	batch size :  50
	length     :  17
	num batches:  1



All mis-matched entries have been validated!
Getting records to update from database...

Creating Table Generator:
	batch size :  50
	length     :  50
	num batches:  2
Updating records ...



Updates complete!  50 records updated.


In [158]:
# Update mismatched artists in the Artists table
# update_mismatched_artists(mismatched_artists, TABLE=Artists) # updated 11/7

Validating mis-mathced entries...
Creating List Generator:
	batch size :  50
	length     :  17
	num batches:  1
Creating List Generator:
	batch size :  50
	length     :  17
	num batches:  1



All mis-matched entries have been validated!
Getting records to update from database...

Creating Table Generator:
	batch size :  50
	length     :  0
	num batches:  1
Updating records ...



Updates complete!  0 records updated.


### Add artist_popularity and track_popularity
We will add values from artist and track popularity from values retrieved from the 
Sportify API.  Since this may change, we will want to make this in such a way that a normal database maintenace can be run to include this task.
